In [ ]:
## RUN THIS CELL IF AND ONLY IF THIS IS OPEN IN GOOGLE COLAB
!git clone https://github.com/ries-lab/SimuFLUX.git
%cd SimuFLUX/python/examples

In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
from scipy import optimize

from python.fluorophores import FlMoveBleach
from python.simulators import SimSequencefile

In [ ]:
## Functions we will need, can be ignored
def getmsd(x,y,time,maxd,dt):
    x = x.squeeze()
    y = y.squeeze()
    time = time.squeeze()
    
    timeint = np.arange(np.min(time), np.max(time)+dt, dt)
    # timeint = np.linspace(np.min(time), np.max(time), (np.max(time)-np.min(time))/dt)
    xint = np.interp(timeint, time, x)
    yint = np.interp(timeint, time, y)
    msd = np.zeros(maxd)
    for di in range(maxd):
        try:
            msd[di] = np.mean((xint[:-di]-xint[di:])**2+(yint[:-di]-yint[di:])**2)
        except ValueError:
            msd[di] = 0

    return msd, dt

def msdfit(msd,dt,linec):
    msdt = np.arange(0, len(msd)*dt+dt, dt)
    ax = plt.gca()
    ax.plot(msdt[1:]/1e3,msd/1e6,linec)
    fp = np.polyfit(msdt[1:], msd, deg=1)
    fpf = np.poly1d(fp)
    ax.plot(msdt/1e3,fpf(msdt)/1e6,linec+"--")
    Dfit = fp[0]/1000/4  # um^2/s
    off = np.sqrt(fp[1])
    ax.set_xlabel('time (s)')
    ax.set_ylabel('MSD ($\mu$m^2)')

    return Dfit, off, msdt


def maxDiffusion(sim, laserpower, repetitions, Ds=None, dname=None, fig=None):
    if Ds is None:
        Ds = np.hstack([0, 0.1, 0.2, np.arange(0.25, 8.25, 0.25)])
    if dname is None:
        dname = "data"
    if fig is None:
        fig = plt.gcf()

    if isinstance(Ds, list):
        Ds = np.array(Ds)

    Ds = Ds.squeeze()
    print(Ds.shape)

    rmse = np.zeros((len(Ds), repetitions, 3))
    efo = np.zeros((len(Ds), repetitions))
    indlosta = np.zeros((len(Ds), repetitions))

    maxerr = 100

    for d in range(len(Ds)):
        for k in range(repetitions):
            sim.posgalvo = [0, 0, 0]
            sim.posEOD = [0, 0, 0]
            sim.time = 0

            fl = FlMoveBleach()
            fl.photonbudget = np.inf
            fl.brightness = 1000*laserpower

            D = Ds[d]  # um^2/s
            fl.makediffusion(D,0.003)
            sim.fluorophores=fl
            out = sim.runSequence(repetitions=1, resetfluorophores=True)
            err = np.sqrt((out.loc.xnm-out.loc.xfl1)**2+(out.loc.ynm-out.loc.yfl1)**2).squeeze()
            print(err.shape)
            try:
                indlost = np.where(err<maxerr)[0][-1]
                filter = (out.loc.itr==np.max(out.loc.itr)).squeeze()
                filter[(indlost+1):] = False
                sr=sim.summarize_results(out,display=False,filter=filter)
                rmse[d,k,:] = sr.rmse
                efo[d,k] = np.nanmean(out.loc.efo[filter])
            except IndexError:
                indlost = 0
                rmse[d,k,:] = np.nan*np.ones(3)
                efo[d,k] = np.nan
            indlosta[d,k] = indlost

    isconverged = indlosta > (sim.sequence['locLimit']-10)  # safe margin
    print(f"isconverged: {isconverged.shape}")
    fconverged = np.sum(isconverged,axis=1).squeeze()/repetitions

    try:
        startind = np.where(fconverged>0.5)[0][-1]
    except IndexError:
        startind = 0
    startD = Ds[startind]
    def fitfun(x, D, alpha):
        return 0.5-special.erf(alpha*(x-D))/2  # sigmoidal
    popt, _ = optimize.curve_fit(fitfun, Ds, fconverged, p0=[startD,1])
    fig, ax = plt.subplots(1,1)
    ax.scatter(Ds, fconverged)
    ax.legend()
    print(f"Ds: {Ds.shape}, fconverged: {fconverged.shape}")
    print(f"fit popt: {popt}")
    Dmax = popt[0]

    indconv = np.where(Ds<=Dmax)[0][-1]
    rmsem = np.nanmean(rmse,axis=1).squeeze()
    efoD0 = np.nanmean(efo[0,:])
    rmseDmax = np.nanmean(rmsem[indconv,:3])

    if np.any(np.array(rmseDmax.shape)!=1):
        rmseDmax = np.nan

    ax = fig.add_subplot(1,3,1)

    hp = ax.plot(Ds, fconverged, label=dname)
    ax.set_ylabel("fraction tracked")
    ax.set_xlabel('Diffusion coefficient um^2/s')
    ax.set_title(f"Dmax: {Dmax:.2f} $\mu$m2/s")
    ax.plot(Ds,fitfun(Ds,*popt), color=hp[0].get_color(), linewidth=1, label=f"{dname}fit")
    ax.legend()

    ax = fig.add_subplot(1,3,2)
    ax.plot(Ds,np.nanmean(efo,axis=1),color=hp[0].get_color(),label=dname)
    ax.set_xlabel('Diffusion coefficient um^2/s')
    ax.set_ylabel("efo kHz")
    ax.legend()

    ax = fig.add_subplot(1,3,3)

    ax.plot(Ds,np.nanmean(rmsem[:,:3],axis=1),color=hp[0].get_color(),label=dname)
    ax.set_ylabel("RMSE of converged (nm)")
    ax.set_xlabel('Diffusion coefficient um^2/s')
    ax.legend()
    
    return Dmax, efoD0, rmseDmax


In [ ]:
sim = SimSequencefile()

In [ ]:
fname = os.path.join(SCRIPT_DIR, "examples", "Tracking_2D.json")
fname2 = os.path.join(SCRIPT_DIR, "settings", "PSFvectorial2D.json") # use a PSF that is defined via a json file
sim.loadsequence(fname, fname2)

In [ ]:
sim.sequence['locLimit'] = 100  # only track for 1000 localizations
sim.makepatterns()
defaultsequence = sim.sequence
defaultdeadtimes = sim.deadtimes
defaultestimators = sim.estimators

In [ ]:
laserpower = 1
updatetime = 0.0025 # ms
repetitions = 20
maxerr = 100

## Plot Example Track

In [ ]:
D = 3.0
sim.posgalvo = [0, 0, 0]
sim.posEOD = [0, 0, 0]
fl=FlMoveBleach()
fl.photonbudget = np.inf
fl.brightness = 1000*laserpower
fl.makediffusion(D,updatetime)
sim.fluorophores = fl
out=sim.runSequence(repetitions=1, resetfluorophores=True)
err = np.sqrt((out.loc.xnm-out.loc.xfl1)**2+(out.loc.ynm-out.loc.yfl1)**2)
indlost = np.where(err<maxerr)[1]

In [ ]:
fig = plt.figure()
ax = plt.subplot(221)
ax.plot(out.loc.loccounter, out.loc.xnm)
ax.plot(out.loc.loccounter,out.loc.xfl1)
ax.plot(out.loc.loccounter, out.loc.ynm)
ax.plot(out.loc.loccounter, out.loc.yfl1)
ax.plot([indlost,indlost],[np.min(out.loc.ynm),np.max(out.loc.ynm)])
ax.set_xlabel('time (localizations)')
ax.set_ylabel('x position (nm)')
ax.legend(['x estimated position','x fluorophore position','y estimated position','y fluorophore position'])

In [ ]:
ax = plt.subplot(222)
ax.plot(out.loc.xnm[:-1], out.loc.ynm[:-1])
tfl = np.arange(0, out.loc.time[-1]+np.min(np.diff(out.loc.time,axis=0)), np.min(np.diff(out.loc.time,axis=0)))
ax.plot(np.interp(tfl,fl.pos[:,0],fl.pos[:,1]),np.interp(tfl,fl.pos[:,0],fl.pos[:,2]))
ax.set_xlabel('x position (nm)')
ax.set_ylabel('y position (nm)')
ax.set_aspect('equal')

## investigate what happens if fluorophore gets lost

In [ ]:
D = 3.0
numtracks = 300
lenwin = 10
averagejump = np.zeros(lenwin)
averageerr = np.zeros(lenwin)
nj = 0
for k in range(numtracks):
    sim.posgalvo = [0, 0, 0]
    sim.posEOD = [0, 0, 0]
    sim.time=0
    fl = FlMoveBleach()
    fl.photonbudget = np.inf
    fl.brightness = 1000*laserpower
    fl.makediffusion(D, updatetime)
    sim.fluorophores = fl
    out=sim.runSequence(repetitions=1, resetfluorophores=True)
    
    err = np.sqrt((out.loc.xnm-out.loc.xfl1)**2+(out.loc.ynm-out.loc.yfl1)**2)
    jx = np.vstack([np.diff(out.loc.xfl1, axis=0),0])
    jy = np.vstack([np.diff(out.loc.yfl1, axis=0),0])
    jump = np.sqrt(jx**2+jy**2)
    
    indlost = np.where(err<maxerr)[0][-1]
    try:
        averagejump += jump[(indlost-lenwin+3):(indlost+3)].squeeze()
        averageerr += err[(indlost-lenwin+3):(indlost+3)].squeeze()
        nj += 1
    except ValueError:
        pass

In [ ]:
ax = plt.subplot(2,2,3)
nx = np.arange(0,len(averageerr)).T-7
ax.plot(nx,averagejump/nj)
ax.plot(nx,averageerr/nj)
ax.set_xlabel('time (localization)')
ax.set_ylabel('jump, localization error (nm)')
ax.legend(['average jump', 'average error'])

## MSD analysis

In [ ]:
tlast = 0
D = 3.0
# plot example track
msdwin, msdwinr = 10, 1000
msdmf = np.zeros(msdwin)
msdfl = np.zeros(msdwin)
msdflr = np.zeros(msdwinr)
imsd, ir = 0, 0
repetitionsmsd = 100
dtmsd, dtmsdr = None, None
for k in range(repetitionsmsd):
    sim.posgalvo = [0, 0, 0]
    sim.posEOD = [0, 0, 0]
    sim.time = 0
    fl = FlMoveBleach()
    fl.photonbudget = np.inf
    fl.brightness = 1000*laserpower
    fl.makediffusion(D,updatetime)
    sim.fluorophores = fl
    out = sim.runSequence(repetitions=1, resetfluorophores=True)
    err = np.sqrt((out.loc.xnm-out.loc.xfl1)**2+(out.loc.ynm-out.loc.yfl1)**2)
    indlost = np.where(err<maxerr)[0][-1]
    
    msdhfr, dtmsdr = getmsd(fl.pos[:,1],fl.pos[:,2],fl.pos[:,0],msdwinr,fl.pos[1,0]-fl.pos[0,0])
    msdflr = msdflr+msdhfr
    ir = ir+1
    
    if indlost > 50:
        time = out.loc.time[:indlost]
        if dtmsd is None:
            dtmsd = np.min(np.diff(time, axis=0))
        xfl = np.interp(time, fl.pos[:,0],fl.pos[:,1])  # should be nearest
        yfl = np.interp(time, fl.pos[:,0],fl.pos[:,2])
        msdhf, dtmsd = getmsd(xfl,yfl,time,msdwin,dtmsd)
        
        msdhm, dtmsd = getmsd(out.loc.xnm[:indlost],out.loc.ynm[:indlost],time,msdwin,dtmsd)
        if not (any(np.isnan(msdhm)) or any(np.isnan(msdfl))):
            msdfl += msdhf
            msdmf += msdhm
            imsd += 1

msdfl=msdfl/imsd
msdmf=msdmf/imsd
msdflr=msdflr/ir

In [ ]:
ax = plt.subplot(224)
Dflr, offflr, _ = msdfit(msdflr,dtmsdr,'m')
Dfl, offfl, _ = msdfit(msdfl,dtmsd,'r')
Dmf, offmf, msdt = msdfit(msdmf,dtmsd,'b')
ax.set_xlim([0, msdt[-1]/1e3])


ax.legend(["fluorophore all",f"D={Dflr:.2f} µm2/s","fluorophore tracked",f"D={Dfl:.2f} µm2/s","Minflux",f"D={Dmf:.2f} µm2/s"])

In [ ]:
fig.tight_layout()

## investigate maximum diffusion coefficient for various conditions

In [ ]:
sim.sequence=defaultsequence # reset
sim.makepatterns()
Dtest = np.hstack([0,0.1, 0.2, np.arange(0.25, 7.25, 0.25)])
# Dtest = [0, 1]

laserpowers = np.hstack([0.05, 0.1, np.arange(0.2, 1.0, 0.2), np.arange(1,5.5,0.5)])

Dmaxl = np.zeros_like(laserpowers)
efoD0l = np.zeros_like(laserpowers)
rmseDmaxl = np.zeros_like(laserpowers)

fig = plt.figure()
for k in range(len(laserpowers)):
    Dmaxl[k], efoD0l[k], rmseDmaxl[k] = maxDiffusion(sim, laserpowers[k], repetitions, Dtest, str(laserpowers[k]))

fig.suptitle('laserpowers')